# HW 1 - Carlos Alvarado

### Problem 1

In [1]:
import os
import pandas as pd
import requests

def get_and_load_data(data_name):
    '''
    Loads "data_name.csv" from local disk, or downloads it if it's not present
    
    Returns: pandas data_frame for "data_name"
    '''
    
    filepath = './data/{}.csv'.format(data_name)
    
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
    else:
        data = helper_download_data(filepath, data_name)
        
    return data
    
def helper_download_data(filepath, data_name):
    offset = 0
    limit = 50000
    response_code = 200
    data = pd.DataFrame({})
    
    while response_code == 200:
        
        url = 'https://data.cityofchicago.org/resource/{}.json?$limit={}&$offset={}'.format(
            data_name, limit, offset)
        print('getting data from', url)
    
        response = requests.get(url)
        response_code = response.status_code
        
        if response_code != 200: 
            print('Failed to download data')
            break
            
        json_data = response.content
    
        data = pd.concat([data, pd.read_json(json_data)])
        
        offset = offset + limit
    
    #save data to csv for future use
    data.to_csv(filepath)
    
    return data
    

In [2]:
SOURCES = [
    ('Graffiti Removal', 'hec5-y4x5'),
    ('Vacant and Abandoned Buildings Reported', '7nii-7srd'),
    ('Pot Holes Reported','7as2-ds3y'),
    ('Sanitation Code Complaints', 'me59-5fac')
]

complaints = pd.DataFrame({})
for comp_type, data_name in SOURCES:
    data = get_and_load_data(data_name)
    print(data.columns)
    data['Complaint Type'] = comp_type
    complaints = pd.concat([complaints, data], ignore_index=True)

Index(['Creation Date', 'Status', 'Completion Date', 'Service Request Number',
       'Type of Service Request', 'What Type of Surface is the Graffiti on?',
       'Where is the Graffiti located?', 'Street Address', 'ZIP Code',
       'X Coordinate', 'Y Coordinate', 'Ward', 'Police District',
       'Community Area', 'SSA', 'Latitude', 'Longitude', 'Location'],
      dtype='object')
Index(['Unnamed: 0', 'address_street_direction', 'address_street_name',
       'address_street_number', 'address_street_suffix',
       'any_people_using_property_homeless_childen_gangs_', 'community_area',
       'date_service_request_was_received',
       'if_the_building_is_open_where_is_the_entry_point_',
       'is_building_open_or_boarded_',
       'is_the_building_currently_vacant_or_occupied_',
       'is_the_building_dangerous_or_hazardous_',
       'is_the_building_vacant_due_to_fire_', 'latitude', 'location',
       'location_of_building_on_the_lot_if_garage_change_type_code_to_bgd_',
       'lon

MemoryError: 

In [ ]:
complaints.loc[10]

In [ ]:
ctcounts = complaints['Complaint Type'].value_counts()
ctcounts

### Problem 2

### Problem 3